In [4]:
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
from KDEpy import FFTKDE
from numpy import trapz

In [11]:
# CFAR version 2, in this slidin window is created 
#on the basis of value of the pixel

class BilateralCFAR_v2(object):

    #initializing the values
    
    def __init__(self,img,tw,gw,bw,pfa,kernel_width):
        self.img = img
        self.kernel_width = kernel_width
        self.tw = tw
        self.gw = gw
        self.bw = bw
        self.pfa = pfa
        self.spatial_component = []
        self.intensity_component = []
        self.combined_component = []
        self.threshold = []
        print("Kernel Ready.")
           
    #checking if the pixel exists
    def isPixelexists(self,size_img,a,b):
        r,c = size_img
        #print(r,c)
        if (a>=0 and a<r) and (b>=0 and b<c) :
            return True
        else:
            return False

    #Computing 4 buffer values.TOP,BOTTOM,LEFT and RIGHT
    def get_topBuffer(self,u,v,size_t,size_g):
        top_buffer = []

        #we have considered the target_window pixels too.
        for p in range(size_t,size_g+1):
                        
            x = u-p
            for m in range(-p,p+1):
                y = v+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    top_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    top_buffer.append(0)

        return top_buffer

    def get_bottomBuffer(self,u,v,size_t,size_g):
        bottom_buffer = []
        
        for p in range(size_t,size_g+1):
            
            x = u+p
            for m in range(-p,p+1):
                y = v+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    bottom_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    bottom_buffer.append(0)

        return bottom_buffer

    def get_leftBuffer(self,u,v,size_t, size_g):
        left_buffer = []
        for p in range(size_t,size_g+1):
            y = v-p
            for m in range(-p,p+1):
                x = u+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    left_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    left_buffer.append(0)

        return left_buffer

    def get_rightBuffer(self,u,v,size_t,size_g):
        right_buffer = []
        
        for p in range(size_t,size_g+1):
            y = v+p
            for m in range(-p,p+1):
                x = u+m
                #print(x,y)
                if self.isPixelexists(self.img.shape,x,y):
                    #print("Found")
                    right_buffer.append(self.img[x][y])
                else:
                    #print("Not found")
                    right_buffer.append(0)

        return right_buffer

    
    def computeSpatialnCombined(self):
        dvi = []
        x_spatial = []
        noise_data = []
        print("Computing DVi and Threshold..")
        
        #radius_t = int(self.tw/2)
        radius_t = 0
        radius_g = int(self.bw/2)
        rows = self.img.shape[0]
        cols = self.img.shape[1]
        x_combined = 0.0
        x_val = 0.0
        
        for i in tqdm(range(self.img.shape[0])):
            for j in range(self.img.shape[1]):
                
                if (self.img[i,j]) > 200:
                    #print("hello")
                    win_top_buffer = self.get_topBuffer(i,j,radius_t,radius_g)
                    win_bottom_buffer = self.get_bottomBuffer(i,j,radius_t,radius_g)
                    win_left_buffer = self.get_leftBuffer(i,j,radius_t,radius_g)
                    win_right_buffer = self.get_rightBuffer(i,j,radius_t,radius_g)

                    guard_buffer = np.array(win_top_buffer + win_bottom_buffer + 
                                            win_left_buffer + win_right_buffer)
                    
                    
                    
                    #x_intensity = (self.img[i,j] - guard_buffer.mean())/guard_buffer.std()
                    x_intensity = self.img[i,j]
                    
                    n = len(guard_buffer)
                    minimum = 1000
                    maximum = -1000
                    sum_spatial = 0.0
                    #print(n)
            
                    for v in guard_buffer:
                        #print(v)
                        valspa = np.exp((-(self.img[i,j] - v)**2)/(2*(self.kernel_width**2)))
                        #print("Valspa: ",valspa)
                        sum_spatial += valspa
                        if valspa < minimum:
                            minimum = valspa
                        elif valspa > maximum:
                            maximum = valspa
                          
                    #print("spatial_sum: ",sum_spatial)
                    x_spati = (sum_spatial - minimum)/(maximum - minimum)
                    #print("f_spatial: ",(f_spatial))
                    x_spatial.append(x_spati)
                    x_combined = x_spati*x_intensity*(4/(self.bw**2))
                    #print(x_combined)
                    
#                     target_box = np.array(self.get_topBuffer(i,j,0,radius_t)+
#                                          self.get_bottomBuffer(i,j,0,radius_t)+
#                                          self.get_leftBuffer(i,j,0,radius_t)+
#                                          self.get_rightBuffer(i,j,0,radius_t))

#                     x,y = FFTKDE(kernel="gaussian", bw="silverman").fit(guard_buffer).evaluate()
#                     a,b = FFTKDE(kernel="gaussian", bw="silverman").fit(target_box).evaluate()
#                     plt.plot(x,y)
#                     plt.plot(a,b)
                    #plt.show()
                    dvi.append((x_combined))
                    #noise_data.append((guard_buffer.mean()))
                else:
                    x_spatial.append(0)
                    dvi.append(0)
                    #noise_data.append(0)
        
        self.spatial_component = np.array(x_spatial).reshape(self.img.shape)
        #x_combined = (4/(self.tw**2))*sum(x_combined)
        self.combined_component = np.array(dvi).reshape(self.img.shape)
        #noise_data = (np.array(noise_data))
        #P = np.array(self.compute_scaleFactor()*noise_data).reshape(self.img.shape)
        print("Process completed, DV image and Threshold Image succesfully Computed.\n")
        return self.combined_component,self.spatial_component

    def binary_search(self,arr, val, start,end, data,x_offset): 
  
        mid = int((start+end)/2)
        #print(mid)
        area = trapz(data[mid:],dx=x_offset)
        #print(area)
        #print(abs(area-val))
        if abs(area - val) < 0.001: 
            return mid 
        elif area > val: 
            return self.binary_search(arr, val, mid, end, data,x_offset) 
        elif area < val:
            return self.binary_search(arr, val, start, mid, data,x_offset) 
    
    def computeThreshold(self):
        
        threshold = []
        radius_t = int(self.tw/2)
        radius_g = int(self.bw/2)
        rows = self.img.shape[0]
        cols = self.img.shape[1]
        x_combined = 0.0
        x_val = 0.0
        
        for i in tqdm(range(self.img.shape[0])):
            for j in range(self.img.shape[1]):
                
                if (self.img[i,j]) > 200:
                    #print("hello")
                    win_top_buffer = self.get_topBuffer(i,j,radius_t,radius_g)
                    win_bottom_buffer = self.get_bottomBuffer(i,j,radius_t,radius_g)
                    win_left_buffer = self.get_leftBuffer(i,j,radius_t,radius_g)
                    win_right_buffer = self.get_rightBuffer(i,j,radius_t,radius_g)

                    guard_buffer = np.array(win_top_buffer + win_bottom_buffer + 
                                            win_left_buffer + win_right_buffer)
                    
                    
                    x,y = FFTKDE(kernel="gaussian", bw="silverman").fit(guard_buffer).evaluate()
                    data = np.array(y)
                    arr = np.array(np.arange(len(data)))
                    threshold_index = self.binary_search(arr,self.pfa,0,len(arr)-1,data,x[2]-x[1])
                    
                    threshold.append(x[threshold_index])
                    # Compute the area using the composite trapezoidal rule.
                    

                    # Compute the area using the composite Simpson's rule.
#                     area = simps(value, dx=x[2]-x[1])
#                     print("area =", area)
                    #plt.plot(x,y)
                    #plt.show()
                    
                    #noise_data.append((guard_buffer.mean()))
                else:
                    threshold.append(0)
        
        threshold = np.array(threshold).reshape(self.img.shape)
        return threshold
        
        
    def shipDetection(self):
        final_image = []
        
        x_combined, x_spatial = self.computeSpatialnCombined()
        threshold = self.computeThreshold()
        
        for i in tqdm(range(self.img.shape[0])):
            for j in range(self.img.shape[1]):
                
                if x_combined[i][j] < threshold[i][j]:
                    
                    final_image.append(1)
                else:
                    final_image.append(0) #valid Ships
        
        final_image = np.array(final_image).reshape(self.img.shape)
        print("Binary Image of Ships is Succesfully Generated.\n")
        return final_image, x_combined, x_spatial,threshold
    